### RunnableLambda Embedding 예제

In [1]:
from langchain_openai import ChatOpenAI
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings


physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question. \
The user is from Korean. Answer the question in Korean.

Here is a question:
{query}"""


embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI(model="gpt-3.5-turbo-0125")
    | StrOutputParser()
)

In [2]:
chain.invoke("What's a black hole")

Using PHYSICS


'A black hole is a region in space where gravity is so strong that nothing, not even light, can escape its pull. Black holes are formed when massive stars collapse under their own gravity. They have a boundary called the event horizon, beyond which nothing can escape. Inside the event horizon, the gravitational pull is so strong that it creates a singularity, a point of infinite density. Black holes can come in different sizes, from stellar-mass black holes to supermassive black holes found at the centers of galaxies.'

In [3]:
print(chain.invoke("What's a integral"))

Using MATH
적분이란 무엇인가요?

적분은 미적분학에서 중요한 개념 중 하나로, 함수의 면적을 구하는 것을 의미합니다. 즉, 함수의 그래프와 x축이 이루는 영역의 면적을 계산하는 것입니다. 적분은 함수의 도함수인 미분과 반대되는 개념으로, 미분이 함수의 기울기를 나타내는 것과 달리 적분은 면적을 구하는 것을 나타냅니다. 적분은 수학적 모델링, 물리학, 공학 등 다양한 분야에서 사용되는 중요한 도구입니다.


#### Embedding 유사도 수치 확인

In [4]:
physics_template = "You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know."


question1 = "What's a black hole"
question2 = "What's a integral"
question3 = "Who's a Lionel Messi"


embeddings = OpenAIEmbeddings()
sentences = [physics_template, question1, question2, question3]
prompt_embeddings = embeddings.embed_documents(sentences)


def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]


for i, sentence in enumerate(prompt_embeddings):
    for j, other_sentence in enumerate(prompt_embeddings):
        if i < j:
            print(f"[유사도: {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}")


[유사도: 0.7442] You are a very smart physics professor. You are great at answering questions about physics in a concise and easy to understand manner. When you don't know the answer to a question you admit that you don't know. 	 <=====> 	 What's a black hole
[유사도: 0.7108] You are a very smart physics professor. You are great at answering questions about physics in a concise and easy to understand manner. When you don't know the answer to a question you admit that you don't know. 	 <=====> 	 What's a integral
[유사도: 0.7154] You are a very smart physics professor. You are great at answering questions about physics in a concise and easy to understand manner. When you don't know the answer to a question you admit that you don't know. 	 <=====> 	 Who's a Lionel Messi
[유사도: 0.7702] What's a black hole 	 <=====> 	 What's a integral
[유사도: 0.7495] What's a black hole 	 <=====> 	 Who's a Lionel Messi
[유사도: 0.7307] What's a integral 	 <=====> 	 Who's a Lionel Messi
